#### This assignment may be worked individually or in pairs. Enter your name/names here:
    

In [2]:
#names here
# Garrett Kelley

# Assignment 1: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1151` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of Diabetic Retinopathy.


A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. *Suggestion:* The dataset is substantially big, for the purpose of easy debugging work with a subset of the data and test your decision tree implementation on that.

#### Implementation: 
A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 buckets. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5.
- For binary features [0/1] the threshold value will be 0.5. All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [149]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log2
from math import floor
from random import shuffle

In [4]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [160]:
def get_data(filename):
    data = []
    csv_data = pd.read_csv(filename, header=None).to_numpy()
    print(len(csv_data))
    for row in csv_data:
        data.append(DataPoint(row[19], row[0:18]))

    return data

In [36]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self):    # for debugging purposes
        if self.is_leaf:
            print ('Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree()
            self.right_child.printTree()

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [138]:
def make_prediction(tree_root, data_point):
    node = tree_root
    prediction = None
    while not node.is_leaf:
        if data_point.features[node.feature_idx] >= node.thresh_val:
            node = node.right_child
        else:
            node = node.leaf_child
    
    return node.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [51]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []

    for row in data:
        if row.features[feature_idx] < threshold:
            left_split.append(row)
        else:
            right_split.append(row)

    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [54]:
def calc_entropy(data):
    entropy = 0.0

    data = [DataPoint(1, [1]), DataPoint(0, [1]), DataPoint(0, [1]), DataPoint(1, [1])]
    num_instances = len(data)
    num_class_0 = 0
    num_class_1 = 0
    for row in data:
        if row.label == 0:
            num_class_0 = num_class_0 + 1
        else:
            num_class_1 = num_class_1 + 1
    p_0 = num_class_0/num_instances
    p_1 = num_class_1/num_instances
    entropy = -1*(p_0*log2(p_0) + p_1*log2(p_1))
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [125]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None

    sorted_data = sorted(data, key=lambda x: x.features[feature_idx])
    midpoints = []

    # generate midpoints and calculate impurity of the parent
    for i in range(len(data) - 1):
        midpoints.append((data[i].features[feature_idx] + data[i + 1].features[feature_idx]) / 2)
    midpoints.insert(0, data[0].features[feature_idx] - (midpoints[0] - data[0].features[feature_idx]))
    midpoints.append(data[len(data) - 1].features[feature_idx] + (data[len(data) - 1].features[feature_idx] - midpoints[len(midpoints) - 1]))    

    # calculate impurity of the parent
    p_0 = 0
    p_1 = 0
    for data_point in data:
        if data_point.label == 0:
            p_0 = p_0 + 1
        else:
            p_1 = p_1 + 1

    p_0 = p_0/len(data)
    p_1 = p_1/len(data)
    I_parent = -1*(p_0*log2(p_0) + p_1*log2(p_1))

    # initialize class distrubution
    class_distribution = np.zeros((len(midpoints), 2, 2))
    for data_point in sorted_data:
        if data_point.label == 0:
            class_distribution[0, 0, 1] = class_distribution[0, 0, 1] + 1
        else:
            class_distribution[0, 1, 1] = class_distribution[0, 1, 1] + 1

    # loop through midpoints and calculate entropy dynamically
    # the minimum entropy is also calulated in this loop
    for i in range(1, len(midpoints) - 1):
        if sorted_data[i - 1].label == sorted_data[i].label:
            continue
        if sorted_data[i - 1].label == 0:
            class_distribution[i, 0, 0] = class_distribution[i - 1, 0, 0] + 1
            class_distribution[i, 0, 1] = class_distribution[i - 1, 0, 1] - 1
            class_distribution[i, 1, 0] = class_distribution[i - 1, 1, 0]
            class_distribution[i, 1, 1] = class_distribution[i - 1, 1, 1]
        else:
            class_distribution[i, 0, 0] = class_distribution[i - 1, 0, 0]
            class_distribution[i, 0, 1] = class_distribution[i - 1, 0, 1]
            class_distribution[i, 1, 0] = class_distribution[i - 1, 1, 0] + 1
            class_distribution[i, 1, 1] = class_distribution[i - 1, 1, 1] - 1
        N = len(data)
        N_left = class_distribution[i, 0, 0] + class_distribution[i, 1, 0]
        N_right = class_distribution[i, 0, 1] + class_distribution[i, 1, 1]
        p_0_left = class_distribution[i, 0, 0]/N_left
        p_1_left = class_distribution[i, 1, 0]/N_left
        p_0_right = class_distribution[i, 0, 1]/N_right
        p_1_right = class_distribution[i, 1, 1]/N_right
        I_left = -1*((p_0_left*log2(p_0_left) if p_0_left else 0) + (p_1_left*log2(p_1_left) if p_1_left else 0))
        I_right = -1*((p_0_right*log2(p_0_right) if p_0_right else 0) + (p_1_right*log2(p_1_right) if p_1_right else 0))
        I_children = (N_left/N)*I_left + (N_right/N)*I_right
        info_gain = I_parent - I_children
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_thresh = midpoints[i]

    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [134]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None

    # loop through features and calculate info gain, pick max
    data = [DataPoint(0, [1, 0]), DataPoint(0, [2, 1]), DataPoint(1, [3, 1])]
    best_info_gain = 0.0
    for i in range(len(data[0].features)):
        info_gain, thresh = calc_best_threshold(data, i)
        if info_gain > best_info_gain:
            best_thresh = thresh
            best_feature = i

    return (best_feature, best_thresh)

Q7. Implement the function `createLeafNode` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [143]:
def createLeafNode(data):
    node = TreeNode()
    node.is_leaf = True
    
    p_0 = 0
    p_1 = 0
    for data_point in data:
        if data_point.label == 0:
            p_0 = p_0 + 1
        else:
            p_1 = p_1 + 1
    
    if p_0 > p_1:
        return 0
    else:
        return 1

Q8. Implement the `createDecisionTree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [187]:
def createDecisionTree(data, max_levels):
    print('ha')
    if max_levels == 1:
        return createLeafNode(data)
    else:
        # create new node and identify best attribute test condition
        node = TreeNode()
        best_feature, best_thresh = identify_best_split(data)
        node.feature_idx = best_feature
        node.thresh_val = best_thresh
        
        # find data that belongs to left child
        v_0 = []
        v_1 = []
        for data_point in data:
            if data_point.features[best_feature] < best_thresh:
                v_0.append(data_point)
            else:
                v_1.append(data_point)
        
        # create left and right children
        node.left_child = createDecisionTree(v_0, max_levels - 1)
        node.right_child = createDecisionTree(v_1, max_levels - 1)
            
    return node

Q9. Given a test set, the function `calcAccuracy` returns the accuracy of the classifier. You'll use the `makePrediction` function for this.

In [180]:
def calcAccuracy(tree_root, data):
    correct = 0
    
    for data_point in data:
        if data_point.label == make_prediction(tree_root, data_point):
            correct = correct + 1

    return correct/len(data)

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [188]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

for i in range(0, 1):
    # partition data into train_set and test_set
    test_set_begin = floor((i / 5) * len(d))
    test_set_end = floor(((i + 1) / 5) * len(d))
    test_set = d[test_set_begin : test_set_end]
    train_set = d[0 : test_set_begin] + d[test_set_end: len(d)]
    
    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    start = time.time()
    tree = createDecisionTree(train_set, 10)
    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calcAccuracy(tree, test_set)
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    tree.printTree()




1151
Training set size: 921
Test set size    : 230
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
ha
h